In [1]:
import pandas as pd
import datetime
import numpy as np

In [49]:
def cal_yoy(df, col):
    return df[col] / df[col + "_prev"] - 1

In [61]:
# State Name Mapping
df_states = pd.read_csv("proc_data/statename_mapping.csv")

In [3]:
df_mortgage = pd.read_csv("raw_data/MORTGAGE30US.csv")

In [55]:
# Since 2018
df_filtered_mortgage = df_mortgage[173:]
df_filtered_mortgage = df_filtered_mortgage.rename(
    {"DATE": "date", "MORTGAGE30US": "value"}, axis=1
)
df_filtered_mortgage["yrmth"] = df_filtered_mortgage["date"].apply(lambda x: x[:7])
df_filtered_mortgage = df_filtered_mortgage.groupby(["yrmth"])["value"].mean().reset_index()
df_filtered_mortgage = df_filtered_mortgage.rename(
    {"yrmth": "date"}, axis=1
)
df_filtered_mortgage["value"] = round(df_filtered_mortgage["value"] / 100, 4)
df_filtered_mortgage["date"] = df_filtered_mortgage["date"].apply(lambda x: x + "-01")

df_filtered_mortgage["value_prev"] = df_filtered_mortgage["value"].shift(12)

df_filtered_mortgage["yoy"] = cal_yoy(df_filtered_mortgage, "value")


In [57]:
df_filtered_mortgage.to_csv("proc_data/MORTGAGE30US_FIL.csv", index=False)


In [38]:
df_filtered_mortgage.head()

,date,value
0,2018-01-01,0.0403
1,2018-02-01,0.0433
2,2018-03-01,0.0444
3,2018-04-01,0.0447
4,2018-05-01,0.0459


## House Inventory

In [96]:
# df_house_inv = pd.read_csv("house_inventory.csv")
df_house_inv = pd.read_csv("raw_data/Metro_invt_fs_uc_sfrcondo_sm_month.csv")


In [113]:
df_house_inv_T = df_house_inv.T

df_house_inv_proc = pd.DataFrame()

for col in df_house_inv_T.columns:
    ts = df_house_inv_T[col][5:]
    df_ts = ts.to_frame().reset_index()
    df_ts.columns = ["date", "inventory"]
    df_ts["PostalCode"] = df_house_inv_T[col]["StateName"]
    # df_ts["inventory_prev"] = df_ts["inventory"].shift(12)
    # df_ts["yoy"] = df_ts["inventory"] / df_ts["inventory_prev"] - 1

    df_house_inv_proc = df_house_inv_proc.append(df_ts)


In [114]:
# df_house_inv_proc["state"] = df_house_inv_proc["state"].fillna("United States")
df_house_inv_proc = df_house_inv_proc.merge(
    df_states, how="left")
df_house_inv_proc["StateName"] = df_house_inv_proc["StateName"].fillna(
    "United States")
df_house_inv_proc = df_house_inv_proc.rename({
    "inventory": "value",
}, axis=1)[["date", "value", "StateName"]]
df_house_inv_proc["value"] = df_house_inv_proc["value"].astype(float)


In [115]:
df_house_inv_proc = df_house_inv_proc.groupby(
    ["date", "StateName"])["value"].sum().reset_index()
df_house_inv_proc = df_house_inv_proc.sort_values(["StateName", "date"])
df_house_inv_proc["value_prev"] = df_house_inv_proc.groupby(["StateName"])[
    "value"].shift(12)
df_house_inv_proc["yoy"] = cal_yoy(df_house_inv_proc, "value")


In [116]:
df_house_inv_proc.to_csv("proc_data/house_inventory.csv", index=False)


## ZORI

In [167]:
df_zori = pd.read_csv("raw_data/Metro_ZORI_AllHomesPlusMultifamily_Smoothed.csv")


In [168]:
df_zori_T = df_zori.T

df_zori_proc = pd.DataFrame()

for col in df_zori_T.columns:
    ts = df_zori_T[col][51:]
    df_ts = ts.to_frame().reset_index()
    df_ts.columns = ["date", "value"]
    df_ts["PostalCode"] = df_zori_T[col]["RegionName"].split(",")[-1].strip()
    df_ts["date"] = df_ts["date"] + "-01"
    # df_ts["inventory_prev"] = df_ts["inventory"].shift(12)
    # df_ts["yoy"] = df_ts["inventory"] / df_ts["inventory_prev"] - 1

    df_zori_proc = df_zori_proc.append(df_ts)

df_zori_proc["value"] = df_zori_proc["value"].astype(int)


In [156]:
# df_zori_proc["state"] = df_zori_proc["state"].fillna("United States")
df_zori_proc = df_zori_proc.merge(
    df_states, how="left")
df_zori_proc["StateName"] = df_zori_proc["StateName"].fillna(
    "United States")
df_zori_proc = df_zori_proc.rename({
    "inventory": "value",
}, axis=1)[["date", "value", "StateName"]]
df_zori_proc["value"] = df_zori_proc["value"].astype(float)


In [157]:
df_zori_proc = df_zori_proc.groupby(
    ["date", "StateName"])["value"].mean().reset_index()
df_zori_proc = df_zori_proc.sort_values(["StateName", "date"])
df_zori_proc["value_prev"] = df_zori_proc.groupby(["StateName"])[
    "value"].shift(12)
df_zori_proc["yoy"] = cal_yoy(df_zori_proc, "value")


In [158]:
df_zori_proc.to_csv("proc_data/zori.csv", index=False)


## House Value ZHVI

In [159]:
df_zhvi = pd.read_csv("raw_data/Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")


In [162]:
df_zhvi_T = df_zhvi.T

df_zhvi_proc = pd.DataFrame()

for col in df_zhvi_T.columns:
    ts = df_zhvi_T[col][221:]
    df_ts = ts.to_frame().reset_index()
    df_ts.columns = ["date", "value"]
    df_ts["PostalCode"] = df_zhvi_T[col]["RegionName"].split(",")[-1].strip()
    df_ts["date"] = df_ts["date"]
    # df_ts["inventory_prev"] = df_ts["inventory"].shift(12)
    # df_ts["yoy"] = df_ts["inventory"] / df_ts["inventory_prev"] - 1

    df_zhvi_proc = df_zhvi_proc.append(df_ts)

df_zhvi_proc = df_zhvi_proc[~df_zhvi_proc["value"].isnull()]
df_zhvi_proc["value"] = df_zhvi_proc["value"].astype(int)


In [165]:
# df_zhvi_proc["state"] = df_zhvi_proc["state"].fillna("United States")
df_zhvi_proc = df_zhvi_proc.merge(
    df_states, how="left")
df_zhvi_proc["StateName"] = df_zhvi_proc["StateName"].fillna(
    "United States")
df_zhvi_proc = df_zhvi_proc.rename({
    "inventory": "value",
}, axis=1)[["date", "value", "StateName"]]
df_zhvi_proc["value"] = df_zhvi_proc["value"].astype(float)

df_zhvi_proc = df_zhvi_proc.groupby(
    ["date", "StateName"])["value"].mean().reset_index()
df_zhvi_proc = df_zhvi_proc.sort_values(["StateName", "date"])
df_zhvi_proc["value_prev"] = df_zhvi_proc.groupby(["StateName"])[
    "value"].shift(12)
df_zhvi_proc["yoy"] = cal_yoy(df_zhvi_proc, "value")


In [171]:
df_zhvi_proc.to_csv("proc_data/zhvi.csv", index=False)
